# Wind power estimation

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
# Time
start = "1990-09"
stop = "2024-11"

# Locations
latitudes = [70.63]
longitudes = [22.46]
assert len(latitudes) == len(longitudes)

# Domains
carra_domains = ["east", "west"]

## Define request

In [ ]:
carra_id = "reanalysis-carra-height-levels"
carra_request = {
    "variable": ["wind_speed"],
    "height_level": ["75_m", "100_m"],
    "product_type": "analysis",
    "time": [f"{hour:02d}:00" for hour in range(0, 24, 3)],
    "data_format": "grib",
}

era5_id = "reanalysis-era5-single-levels"
era5_request = {
    "product_type": ["reanalysis"],
    "variable": ["100m_u_component_of_wind", "100m_v_component_of_wind"],
    "time": [f"{hour:02d}:00" for hour in range(24)],
    "area": [72, 20, 68, 24],
    "data_format": "grib",
    "download_format": "unarchived",
}

## Define functions to cache

In [ ]:
def find_nearest_locations(ds, latitudes, longitudes):
    (varname,) = set(ds.data_vars)
    ds = ds.rename({varname: "ws"}).expand_dims("height")
    for coord in ("longitude", "latitude"):
        ds[f"nearest_{coord}"] = ds[coord]
        ds[f"nearest_{coord}"].attrs = {}
    grid = xr.Dataset(
        coords={
            "longitude": ("location", latitudes, {"standard_name": "longitude"}),
            "latitude": ("location", longitudes, {"standard_name": "latitude"}),
        }
    )
    return diagnostics.regrid(ds, grid, "nearest_s2d", locstream_out=True)

## Download and transform

In [ ]:
datasets = []
for domain in carra_domains:
    print(f"{domain = }")
    request = download.update_request_date(
        carra_request | {"domain": f"{domain}_domain"},
        start,
        stop,
        stringify_dates=True,
    )
    ds = download.download_and_transform(
        carra_id,
        request,
        chunks={"year": 1, "height_level": 1},
        transform_func=find_nearest_locations,
        transform_func_kwargs={"latitudes": latitudes, "longitudes": longitudes},
        drop_variables=["step"],
    )
    datasets.append(ds.expand_dims(domain=[domain]))
ds = xr.concat(datasets, "domain")

## Quick and dirty plot

In [ ]:
ds["ws"].squeeze("location").plot(col="domain", row="height")